In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
import math
import numpy as np
from glob import glob

In [2]:
images = [image.load_img(p, target_size=(224, 224))
          for p in glob('cats_dogs/cats/*jpg') + glob('cats_dogs/dogs/*jpg')]
vector = np.asarray([image.img_to_array(img) for img in images])

In [3]:
vector.shape

(2000, 224, 224, 3)

In [4]:
base_model = VGG16(weights='imagenet', include_top=True)

In [5]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
vectors = model.predict(preprocess_input(vector))

In [10]:
vectors.shape

(2000, 4096)

In [11]:
labels = [1] * 1000 + [0] * 1000

In [10]:
print(labels)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [12]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(vectors, labels, test_size=0.2)
# cats => 1, dogs => 0

In [13]:
from sklearn.linear_model import LogisticRegression
lr_p1 = LogisticRegression(C=0.1, penalty='l1', solver='saga', max_iter=10000) 
lr_p1.fit(X_train, y_train)
lr_p1.score(X_test, y_test)

0.98

In [10]:
lr_p2 = LogisticRegression(C=10, penalty='l2', solver='saga', max_iter=500) 
lr_p2.fit(X_train, y_train)
lr_p2.score(X_test, y_test)

C:\Users\Sang\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.9225

# 다른 전통적인 기계학습 classifiers

In [32]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train) 
clf.score(X_test, y_test)

C:\Users\Sang\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.8475

In [33]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(max_depth=2, n_estimators=1000)
xgb_model.fit(X_train, y_train)
xgb_model.score(X_test, y_test)

0.87

## 다른 층의 정보를 feature 정보로 사용해 보기

### fc1 층의 출력값 사용하기

In [13]:
model1 = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)

In [14]:
vectors1 = model1.predict(vector/255)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(vectors1, labels, test_size=0.2)

In [16]:
lr1_p1 = LogisticRegression(C=10, penalty='l1', solver='saga', max_iter=500) 
lr1_p1.fit(X_train, y_train)
lr1_p1.score(X_test, y_test)

C:\Users\Sang\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.94

### Bottleneck feature 사용해보기

In [17]:
model2_base = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

In [18]:
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential

In [19]:
model2 = Sequential()
model2.add(model2_base)
model2.add(GlobalAveragePooling2D())

In [20]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


In [21]:
vectors2 = model2.predict(vector/255)

In [22]:
vectors2.shape

(2000, 512)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(vectors2, labels, test_size=0.2)

In [27]:
lr2_p1 = LogisticRegression(C=10, penalty='l1', solver='saga', max_iter=1000) 
lr2_p1.fit(X_train, y_train)
lr2_p1.score(X_test, y_test)

C:\Users\Sang\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.9075